In [6]:
import pandas as pd
in_out_df = pd.read_csv('aac_intakes_outcomes.csv')
in_df = pd.read_csv('aac_intakes.csv')
out_df = pd.read_csv('aac_outcomes.csv')

In [8]:
# Clean data
in_df['datetime'] = pd.to_datetime(in_df['datetime'])

### Is there an area where more pets are found?
#### Find the top 5 places where animals are found so the shelter can coordinate with local volunteers and animal control to monitor these areas.

Most pets are found in Austin, TX. 

To find this answer I used the "found_location" column in the intakes/outcomes csv file to find counts for each location. I then printed the top five locations, sorted in decending order by count.

In [5]:
location_counts = in_out_df['found_location']
top_five_locations = location_counts.value_counts()[:5]

print("Top 5 Locations where pets are found:")
top_five_locations

Top 5 Locations where pets are found:


found_location
Austin (TX)                          14311
Outside Jurisdiction                   945
Travis (TX)                            907
7201 Levander Loop in Austin (TX)      514
Del Valle (TX)                         407
Name: count, dtype: int64

### What is the average number of pets found in a month in the year 2015? Are there months where there is a higher number of animals found?
#### Knowing the number of pets the shelter might see in a month can help them gather enough resources and donations to care for the animals they receive.

The average number of pets found per month in 2015 was 1,559. More pets than average were found from May until October.

I found this answer by using the "datetime" column in the intakes file where the year was equal to 2015. I then filtered by month and used value_counts().sort_index() to organize the count by month, then used mount_counts.mean() to determine the average for the year. To find the months where there was a higher number of intakes I filterd the monthly counts to those that are over 1,559.

In [17]:
pets_2015 = in_df[in_df['datetime'].dt.year == 2015].copy()
pets_2015['month'] = pets_2015['datetime'].dt.month
monthly_counts = pets_2015['month'].value_counts().sort_index()

average_pets_per_month = monthly_counts.mean()

average_pets_per_month

1559.3333333333333

In [18]:
monthly_counts

month
1     1198
2     1119
3     1346
4     1543
5     2094
6     2189
7     1635
8     1718
9     1591
10    1740
11    1411
12    1128
Name: count, dtype: int64

In [19]:
months_over_1559 = monthly_counts[monthly_counts > 1559]
months_over_1559

month
5     2094
6     2189
7     1635
8     1718
9     1591
10    1740
Name: count, dtype: int64

### What is the ratio of incoming pets vs. adopted pets?
#### This key metric helps the shelter know how they are doing.

The ratio of incoming vs adopted pets is 2.34.

To find this answer I found the number of pets in the intake file, then found the number of pets in the outcomes file where their "outcome_type" equals "adoption". Then divided the number of incoming pets by the number that were adopted.

In [21]:
total_incoming_pets = len(in_df)

adopted_pets = out_df[out_df['outcome_type'] == 'Adoption']
total_adopted_pets = len(adopted_pets)

adoption_ratio = total_incoming_pets / total_adopted_pets

adoption_ratio

2.3424573498480954

### What is the distribution of the types of animals in the shelter?
#### Find the count of each type of animal in the shelter.

There are 45,743 dogs, 29,659 cats, 4,434 "other", 342 birds, and 9 animals in the livestock catergory.

To find this answer I used the "animal_types" column to count the number of animals in each category. In addition I found the percentage of each category; dogs made up 57.05%, cats made up 36.99%, etc.

In [22]:
animal_type_counts = in_df['animal_type'].value_counts()
animal_type_counts

animal_type
Dog          45743
Cat          29659
Other         4434
Bird           342
Livestock        9
Name: count, dtype: int64

In [23]:
animal_type_distribution = (animal_type_counts / animal_type_counts.sum()) * 100

animal_type_distribution

animal_type
Dog          57.045406
Cat          36.987292
Other         5.529575
Bird          0.426503
Livestock     0.011224
Name: count, dtype: float64

### What are the adoption rates for specific breeds?
#### Find the top 5 dog breeds in the shelter (based on count) and then find the adoption percentage of each breed.

The top five breeds, as well as their adoption percentages, are Pit Bull Mix (37.32%), Chihuahua Shorthair Mix (47.18%), Labrador Retriever Mix (49.66%), German Shepherd Mix (47.73%), and Australian Cattle Dog Mix (56.02%).

To find this I used the "animal_type" and "breed" columns to find the number of each breed that were listed in the intake file, and then displayed the first five in descending order. I then created a 'for' loop to find the percentages of each breed in the "top_5_breeds_list".

In [26]:
dogs_intake = in_df[in_df['animal_type'] == 'Dog']
dogs_outcome = out_df[out_df['animal_type'] == 'Dog']
top_5_dog_breeds = dogs_intake['breed'].value_counts().head(5)

top_5_dog_breeds

breed
Pit Bull Mix                 6382
Chihuahua Shorthair Mix      4860
Labrador Retriever Mix       4841
German Shepherd Mix          1963
Australian Cattle Dog Mix    1105
Name: count, dtype: int64

In [27]:
top_5_breeds_list = top_5_dog_breeds.index.tolist()

adoption_percentages = {}

for breed in top_5_breeds_list:
    total_intakes = len(dogs_intake[dogs_intake['breed'] == breed])

    total_adoptions = len(dogs_outcome[(dogs_outcome['breed'] == breed) & (dogs_outcome['outcome_type'] == 'Adoption')])

    adoption_percentage = (total_adoptions / total_intakes) * 100
    adoption_percentages[breed] = round(adoption_percentage, 2)
    
adoption_percentages

{'Pit Bull Mix': 37.32,
 'Chihuahua Shorthair Mix': 47.18,
 'Labrador Retriever Mix': 49.66,
 'German Shepherd Mix': 47.73,
 'Australian Cattle Dog Mix': 56.02}

### What are the adoption rates for different colorings?
#### Find the top 5 colorings in the shelter (based on count) and then find the adoption percentage of each color.

THe top five colorings, as well as their adoption percentages, are black/white (45.73%), black (41.09%), brown tabby (42.66%), brown (22.19%), and white (37.98%).

I set this up in the same way as the pervious question, but used the "color" column rather than "breed". The adoptions calcualtion was set up in the same way.

In [29]:
top_5_colors = in_df['color'].value_counts().head(5)
top_5_colors

color
Black/White    8340
Black          6710
Brown Tabby    4487
Brown          3618
White          2849
Name: count, dtype: int64

In [31]:
top_5_colors_list = top_5_colors.index.tolist()

color_adoption_percentages = {}

for color in top_5_colors_list:
    total_intakes_color = len(in_df[in_df['color'] == color])

    total_adoptions_color = len(out_df[(out_df['color'] == color) & (out_df['outcome_type'] == 'Adoption')])

    adoption_percentage_color = (total_adoptions_color / total_intakes_color) * 100
    color_adoption_percentages[color] = round(adoption_percentage_color, 2)

color_adoption_percentages

{'Black/White': 45.73,
 'Black': 41.09,
 'Brown Tabby': 42.66,
 'Brown': 22.19,
 'White': 37.98}

### About how many animals are spayed/neutered each month?
#### This will help the shelter allocate resources and staff. Assume that all intact males and females will be spayed/neutered.

The average number of animals that are spayed/neutered each month is 914.

I used the "sex_upon_intake" column and found those that were listed as "intact male" or "intact female" to determine which animals need to be spayed/neutered. I then broke the datetime column down by month and year, found the total count of animals needing resources and divided by the number of months.

In [34]:
intact_animals = in_df[in_df['sex_upon_intake'].isin(['Intact Male', 'Intact Female'])].copy()

intact_animals['year'] = intact_animals['datetime'].dt.year
intact_animals['month'] = intact_animals['datetime'].dt.month

spay_neuter_counts = intact_animals.groupby(['year', 'month']).size()

total_neuters = spay_neuter_counts.sum()

num_months = len(spay_neuter_counts)

average_monthly_neuters = total_neuters / num_months

average_monthly_neuters

913.8518518518518

### Extra Credit


### How many animals in the shelter are repeats? Which animal was returned to the shelter the most?

#### This means the animal has been brought in more than once.

6,154 animals were repeats, and a rat terrier mix named "Lil Bit" was returned the most (13 times).

I found the number of times each "animal_id" was listed, then filtered by those that were higher than "1". Then found the "animal_id" that had the highest count, and then displayed the details associated with that "animal_id" (name + breed).

In [40]:
animal_id_counts = in_df['animal_id'].value_counts()

repeat_animals = animal_id_counts[animal_id_counts > 1]

num_repeat_animals = len(repeat_animals)

max_returns = repeat_animals.max()
animal_ids_most_returns = repeat_animals[repeat_animals == max_returns].index

animals_most_returns_details = in_df[in_df['animal_id'].isin(animal_ids_most_returns)][['animal_id', 'name', 'breed']].drop_duplicates()

num_repeat_animals, animals_most_returns_details, max_returns

(6154,
      animal_id     name            breed
 6393   A721033  Lil Bit  Rat Terrier Mix,
 13)


### What are the adoption rates for the following age groups?

    baby: 4 months and less (43.12%)
    young: 5 months - 2 years (50.48%)
    adult: 3 years - 10 years (33.67%)
    senior: 11+ (12.5%)

I defined "categorize_age_group" to classify animals into age groups based on their "age_upon_outcome" data. The created a column titled "age_group" to store the classifications. Then found the adoption rate by dividing the number of adoptions by the total count for each category.

In [42]:
def categorize_age_group(age_upon_outcome):
    if isinstance(age_upon_outcome, str):
        age_split = age_upon_outcome.split()
        age = int(age_split[0])
        unit = age_split[1]

        # Convert age to years if in months or weeks
        if 'month' in unit:
            age = age / 12
        elif 'week' in unit or 'day' in unit:
            age = 0  # Considered baby

        # Categorize based on age
        if age <= 0.33:  # 4 months or less
            return 'baby'
        elif age <= 2:  # 5 months - 2 years
            return 'young'
        elif age <= 10:  # 3 years - 10 years
            return 'adult'
        else:  # 11 years and older
            return 'senior'
    else:
        return 'unknown'

out_df['age_group'] = out_df['age_upon_outcome'].apply(categorize_age_group)

In [43]:
adoptions_df = out_df[out_df['outcome_type'] == 'Adoption']

adoption_counts = adoptions_df['age_group'].value_counts()

total_counts = out_df['age_group'].value_counts()

adoption_rates = (adoption_counts / total_counts) * 100

adoption_rates

age_group
young      43.119363
baby       50.478607
adult      33.673191
senior     21.234454
unknown    12.500000
Name: count, dtype: float64

### If spay/neuter for a dog costs 100 and a spay/neuter for a cat costs 50, how much did the shelter spend in 2015 on these procedures?

The total cost in 2015 was $863,250.

I filtered the intakes data to find the number of dogs and cats that were intact, then calcualting the cost based on the total counts. Dogs cost 100 and cats cost 50 in this example.

In [44]:
intakes_2015 = in_df[in_df['datetime'].dt.year == 2015]

intact_dogs_2015 = intakes_2015[(intakes_2015['animal_type'] == 'Dog') & 
                                (intakes_2015['sex_upon_intake'].isin(['Intact Male', 'Intact Female']))]
intact_cats_2015 = intakes_2015[(intakes_2015['animal_type'] == 'Cat') & 
                                (intakes_2015['sex_upon_intake'].isin(['Intact Male', 'Intact Female']))]

num_intact_dogs = len(intact_dogs_2015)
num_intact_cats = len(intact_cats_2015)

cost_dog = 100
cost_cat = 50

total_cost = (num_intact_dogs * cost_dog) + (num_intact_cats * cost_cat)

total_cost

863250